In [1]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input


/Users/cricket/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import numpy as np
#from utils import *
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Model

In [3]:
batch_size=32
num_classes = 120
image_size = 224
num_channels = 3

In [22]:
train_datagen = ImageDataGenerator(rotation_range=40,
                                   shear_range=0.2,
                                   #channel_shift_range=20,
                                   #width_shift_range=0.2,
                                   #height_shift_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('../data/cropped/train',
                                                    target_size=(224,224),
                                                    class_mode='categorical',
                                                    batch_size=batch_size)

validation_generator = test_datagen.flow_from_directory('../data/cropped/test', 
                                                        target_size=(224,224),
                                                        class_mode='categorical',
                                                        batch_size=batch_size)

Found 12000 images belonging to 120 classes.
Found 8580 images belonging to 120 classes.


# Complete VGG16

In [ ]:
model = VGG16()

In [ ]:
img = load_img('../data/cropped/test/n02094258-Norwich_terrier/n02094258_992.jpg', target_size=(224, 224))  # Charger l'image
img = img_to_array(img)  # Convertir en tableau numpy
img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))  # Créer la collection d'images (un seul échantillon)
img = preprocess_input(img)  # Prétraiter l'image comme le veut VGG-16

In [ ]:
y = model.predict(img)  # Prédir la classe de l'image (parmi les 1000 classes d'ImageNet)

In [ ]:
from keras.applications.vgg16 import decode_predictions

# Afficher les 3 classes les plus probables
print('Top 3 :', decode_predictions(y, top=3)[0])

# Transfer learning

In [30]:
# Charger VGG-16 pré-entraîné sur ImageNet et sans les couches fully-connected
model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

#model.summary()

In [31]:
# Récupérer la sortie de ce réseau
x = model.output



In [32]:
# Ajouter la nouvelle couche fully-connected pour la classification à 10 classes
predictions = Dense(train_generator.num_classes, activation='softmax')(x)



In [33]:
# Définir le nouveau modèle
new_model = Model(inputs=model.input, outputs=predictions)

In [34]:
for layer in new_model.layers:
    layer.trainable = False

In [35]:
# Compiler le modèle
from keras import optimizers
new_model.compile(loss="categorical_crossentropy",
                  #optimizer=Adam(lr=0.0001),
                  optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                  metrics=["accuracy"])

# Entraîner sur les données d'entraînement (X_train, y_train)
new_model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)

Epoch 1/20


ValueError: Error when checking target: expected dense_3 to have 4 dimensions, but got array with shape (32, 120)

In [29]:
train_generator.num_classes

120

In [12]:
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [ ]:
#base_model = VGG16(include_top=False).model
base_model = Vgg16BN(include_top=False).model
# add a global spatial average pooling layer
x = base_model.output
x = Flatten()(x)
x = Dropout(0.4)(x)
# let's add two fully-connected layer
x = Dense(2048, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Dense(2048, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(num_classes, activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_generator, samples_per_epoch=train_generator.nb_sample, nb_epoch=10, 
                        validation_data=validation_generator, nb_val_samples=validation_generator.nb_sample)

In [ ]:
model.save_weights('vgg_model_weights.h5')